In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")

model.eval()

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

c:\Users\ayush\OneDrive\Desktop\alphagaze\alphagaze_env\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ayush\.cache\huggingface\hub\models--yiyanghkust--finbert-tone. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: yiyanghkust/finbert-tone
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30873, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [6]:
news_df = pd.read_csv("C:\\Users\\ayush\\OneDrive\\Desktop\\alphagaze\\data\\IN-FINews Dataset.csv")

news_df["Date"] = pd.to_datetime(news_df["Date"])
news_df.head()

,Title,Date,Description,Author,Content,Keywords,URL
0,"Aurobindo Pharma, Dr Reddy's, Lupin, other pha...",2025-07-08,'A renewed push for 'Buy America' could compli...,Debaroti Adhikary,The shares of Indian pharma companies dropped ...,"pharma stocks, Nifty Pharma, Aurobindo Pharma ...",https://www.moneycontrol.com/news/business/mar...
1,Bharti Airtel share price falls 1.53%; stock a...,2025-07-18,"With the stock currently trading at Rs 1,900.4...",Alpha Desk,Shares of Bharti Airtel were trading lower at ...,"Bharti Airtel, shares, stock price, Nifty 50, ...",https://www.moneycontrol.com/news/business/sto...
2,Indian Oil and Adani Gas JV plans $161 million...,2025-07-09,The city gas distribution firm’s deal could be...,Bloomberg,IndianOil-Adani Gas Pvt. is planning to raise ...,"IndianOil-Adani Gas, IOC, convertible bond sale",https://www.moneycontrol.com/news/business/com...
3,Cipla Shares Fall in Today's Trading Session A...,2025-06-03,"The stock's last traded price is Rs 1,461.30, ...",Alpha Desk,"During today's trading session, Cipla expe...","Cipla, stock market, financial performance, re...",https://www.moneycontrol.com/news/business/sto...
4,Here are the sectors and stocks in focus amids...,2025-02-07,With the recent liquidity injection by the Res...,Anishaa Kumar,"Sectors such as consumer durables, financials,...","MPC, RBI, stocks",https://www.moneycontrol.com/news/business/sto...


In [9]:
# Keep only Date and Title
news_df = news_df[["Date", "Title"]]

# Rename for consistency
news_df = news_df.rename(columns={
    "Date": "ds",
    "Title": "News"
})

# Convert to datetime
news_df["ds"] = pd.to_datetime(news_df["ds"])

# Drop empty headlines
news_df = news_df.dropna(subset=["News"])
news_df = news_df[news_df["News"].str.strip() != ""]

news_df.head()


,ds,News
0,2025-07-08,"Aurobindo Pharma, Dr Reddy's, Lupin, other pha..."
1,2025-07-18,Bharti Airtel share price falls 1.53%; stock a...
2,2025-07-09,Indian Oil and Adani Gas JV plans $161 million...
3,2025-06-03,Cipla Shares Fall in Today's Trading Session A...
4,2025-02-07,Here are the sectors and stocks in focus amids...


In [11]:
def finbert_sentiment(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=256
    )
    
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    
    positive = probs[0][0].item()
    negative = probs[0][1].item()
    
    return positive - negative


In [12]:
news_df["sentiment"] = news_df["News"].apply(finbert_sentiment)

news_df.head()


,ds,News,sentiment
0,2025-07-08,"Aurobindo Pharma, Dr Reddy's, Lupin, other pha...",-0.000096
1,2025-07-18,Bharti Airtel share price falls 1.53%; stock a...,0.824368
2,2025-07-09,Indian Oil and Adani Gas JV plans $161 million...,0.999842
3,2025-06-03,Cipla Shares Fall in Today's Trading Session A...,0.983879
4,2025-02-07,Here are the sectors and stocks in focus amids...,0.994460


In [13]:
daily_sentiment = (
    news_df
    .groupby("ds")["sentiment"]
    .mean()
    .reset_index()
)

daily_sentiment.head()


,ds,sentiment
0,2025-02-07,0.605220
1,2025-02-10,0.025723
2,2025-02-11,0.419806
3,2025-02-12,0.481427
4,2025-02-13,0.760332


In [14]:
daily_sentiment.to_csv("C:\\Users\\ayush\\OneDrive\\Desktop\\alphagaze\\data\\daily_sentiment.csv", index=False)